In [1]:
import lyricsgenius
import os
from dotenv import load_dotenv
load_dotenv()

True

## Fetch Data

In [ ]:
os.chdir("./data/raw/")

In [ ]:
offset = 116
clientID = os.getenv("clientID")
clientAccessToken = os.getenv("clientAccessToken")
clientSecret = os.getenv("clientSecret")

In [ ]:
def fetchAndSave(artist, genius):
    print("search for " + artist)
    artist = genius.search_artist(artist, max_songs=500, per_page= 50, sort="popularity", get_full_info= False)
    print("saving lyrics")
    artist.save_lyrics()


In [ ]:
with open("../../topRappers.txt", "r") as fp:
    genius = lyricsgenius.Genius(client_access_token = clientAccessToken, verbose=False, timeout = 15)
    genius.excluded_terms = ["(Remix)", "(Live)"]
    count = 0
    for i, artist in enumerate(fp):
        # print(artist)
        # print (i)
        if i >=  offset :
            try:
                fetchAndSave(artist, genius)
            except TimeoutError:
                continue

## Clean Data

In [19]:
import json
files = os.listdir("./data/raw/")

In [21]:
ignorelist = ["Lyrics will be available" , "Cover Art", "Lyrics for this song have yet to be released", "Lyrics will be put on this page once the song is released",
"Chance the Rapper Logo", "Someone add the lyrics", ]

In [22]:
for filename in files:
    # temp = ""
    # filename = "Lyrics_Dax.json"
    with open(f'./data/raw/{filename}') as f:
        data = json.load(f)
        song_obj = {}
        artist = data["name"]
        song_obj["artist"] = artist

        song_obj["songs"] = []
        for song in data['songs']:
            lyrics = song['lyrics']
            if "My Krazy Life Tour" in song["title"]:
                continue
            if any(ignore in lyrics for ignore in ignorelist):
                continue
            if len(lyrics)<20:
                if "Soon" in lyrics:
                    continue
            # lyrics.decode("utf-8")
            # print(lyrics)
            # temp = lyrics
            song_obj["songs"].append(song['lyrics'])
        with open(f'./data/cleaned/{artist}.json', 'w', encoding='utf-8') as outfile:
            json.dump(song_obj, outfile)

In [30]:
import re
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
import math

files = os.listdir("./data/cleaned/")
chorusCount = 0
verseCount = 0
songCount = 0
# chorusRE = re.compile('\[.*chorus.*\]|(.*chorus.*)|Chorus.*:|{.*chorus.*}|\[.*interlude\]|\[Chorus\]|\[Intro\]|\[hook.*\]|\[outro.*\]|\[...*\]', re.IGNORECASE)
chorusRE = re.compile('(.*chorus.*)|Chorus.*:|{.*chorus.*}|\[...*\]', re.IGNORECASE)

verseRE = re.compile("\[.*verse.*\]|(.*verse.*)|verse*:|{.*verse.*}", re.IGNORECASE)
# print("reset")
longerCount = 0
for filename in files:
    with open(f'./data/cleaned/{filename}', encoding = "utf-8") as f:
        data = json.load(f)
        songs = data["songs"]
        for song in songs:
            encoding = tokenizer.encode(song)
            if len(encoding) > 1023:
                splits = song.split("\n\n")
                middle = math.floor((len(splits) - 1) / 2)
                first = ""
                second = ""
                for i in range (middle):
                    first += splits[i] + "\n\n"
                for i in range(middle, len(splits)):
                    second += splits[i] + "\n\n"
                half1 = tokenizer.encode(first)
                half2 = tokenizer.encode(second)

                if len(half1) > 1023 or len(half2) > 1023:
                    longerCount +=1

                

            # # print(song)
            # choir = chorusRE.findall(song)
            # verses = verseRE.findall(song)

            # if len(choir) > 0 and len(verses) >0:
            #     chorusCount += 1
            #     verseCount +=1
            #     # chorus += len(choir)
            # elif len(verses) >0:
            #     verseCount +=1
            # elif len(choir)>0:
            #     chorusCount += 1
            # else:
            #     if len(song) < 500:
            #         print(song)
            #     # print(len(song))
            #     break
            # songCount +=1
# print(chorusCount)
# print(songCount)
# print(verseCount)

print(longerCount)

1043
